<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/0.51%EA%B9%8C%EC%A7%80%20%EA%B0%9C%EC%84%A0%EC%8B%9C%ED%82%A8%20%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [3]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install catboost


In [5]:
!pip install xgboost lightgbm


In [22]:
import pandas as pd
import numpy as np
import os

# 경로 함수 정의
def get_path(filename):
    return "/content/open_1/" + filename

# 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))

# target 분리
y = train['Cancer']
X = train.drop(columns=['ID', 'Cancer'])
X_test = test.drop(columns=['ID'])


In [23]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

categorical_cols = X.select_dtypes(include='object').columns
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    le.fit(X[col])
    X[col] = le.transform(X[col])

    X_test[col] = X_test[col].map(lambda s: '<UNK>' if s not in le.classes_ else s)
    le.classes_ = np.append(le.classes_, '<UNK>')
    X_test[col] = le.transform(X_test[col])

    encoders[col] = le


In [24]:
def build_stats(df):
    stats = {}
    for col in df.columns:
        if df[col].dtype != 'object' and col != 'Cancer':
            stats[col] = {
                'mean': df[col].mean(),
                'std': df[col].std(),
                'min': df[col].min(),
                'max': df[col].max()
            }
    return stats

def add_41_features(df, stats):
    df_new = df.copy()
    for col, stat in stats.items():
        if col in df.columns:
            df_new[f'{col}_norm'] = (df[col] - stat['mean']) / stat['std']
            df_new[f'{col}_is_outlier'] = ((df[col] < stat['mean'] - 2 * stat['std']) |
                                           (df[col] > stat['mean'] + 2 * stat['std'])).astype(int)
            df_new[f'{col}_range'] = stat['max'] - stat['min']
    return df_new

stats = build_stats(pd.concat([X, X_test]))
X_derived = add_41_features(X, stats)
X_test_derived = add_41_features(X_test, stats)


In [25]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import lightgbm as lgb

# SMOTE 적용
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_derived, y)

# LGBM 모델 훈련
model = lgb.LGBMClassifier(random_state=42)
model.fit(X_smote, y_smote)

# 예측 및 threshold 최적화
y_probs = model.predict_proba(X_derived)[:, 1]
best_thresh = 0.5
best_f1 = 0

for t in np.arange(0.3, 0.7, 0.01):
    y_pred = (y_probs > t).astype(int)
    score = f1_score(y, y_pred)
    if score > best_f1:
        best_f1 = score
        best_thresh = t

print(f'최적 threshold: {best_thresh}, F1 score: {best_f1:.4f}')


[LightGBM] [Info] Number of positive: 76700, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.124369 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4699
[LightGBM] [Info] Number of data points in the train set: 153400, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
최적 threshold: 0.31, F1 score: 0.4875


In [27]:
submission = pd.read_csv(get_path("sample_submission.csv"))

submission['Cancer'] = (model.predict_proba(X_test_derived)[:, 1] > best_thresh).astype(int)
submission.to_csv('submission_final.csv', index=False)


In [28]:
from google.colab import files
files.download('submission_final.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>